In [7]:
from copy import deepcopy
import numpy as np
import pandas as pd
import json, re, nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

import itertools

from nltk.stem.wordnet import WordNetLemmatizer
Lem = WordNetLemmatizer()

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("talk")

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
scopus = pd.read_csv("scopus.csv")
keyword_org_name = "keywords.txt"
keyword_abb_name = "keywords_abb.json"
keyword_dash_name = "keywords_dash.json"
keyword_dict_name = "keywords_dict.json"
keyword_single_name = "keywords_single.txt"
keyword_plural_name = "keywords_plural.json"
unicode_name = "unicode_json"

In [4]:
# unicode dictionary
with open('unicode_dict.json') as j:
    unicode_dict = json.load(j)

In [5]:
keywords = []
line = ''

with open("BIPV_ML_all.txt") as f:
    while line != "EF":
        line = f.readline()
        if line[:3] == "DE ":
            keywords_new = line[3:].rstrip('\n').split(";")
            for keyword_new in keywords_new:
                keywords += [keyword_new.rstrip(' ').lstrip(' ')]

keywords = sorted(keywords)
with open(keyword_org_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords:
        f.write(f"{keyword}\n")

In [6]:
# retreive keywords
keywords_raw_ = np.genfromtxt("keywords.txt", dtype="str", delimiter="\n", encoding='utf-8-sig') #utf-8-sig for \ufeff removal

# get unique ones
keywords_raw = np.unique(keywords_raw_)

# abbriviations detection (key candidates)
keywords_abb = {}
keywords_abb_key_cand = []
for keyword_raw_ in keywords_raw_:
    kwr = keyword_raw_.split()
    for kw in kwr:
        kw = kw.lstrip("(").rstrip(")")
        if (kw == kw.upper()) \
           and (kw.lstrip("(").rstrip(")") not in keywords_abb_key_cand) \
           and (ord(kw[0]) >= ord('A') and ord(kw[0]) <= ord('Z')) \
           and (len(kw) > 1):
            keywords_abb_key_cand.append(kw.lstrip("(").rstrip(")").rstrip(":").rstrip(","))

# convert to lower cases
# keywords_raw = [kw.lower() for kw in keywords_raw_]

# duplicate for values, and convert to lower cases
keywords_value = deepcopy(keywords_raw)

# convert to lower cases
keywords_value = [kw.lower() for kw in keywords_value]

# remove staring characters
keywords_value = [kw.lstrip('-') for kw in keywords_value]
keywords_value = [kw.rstrip('-') for kw in keywords_value]
keywords_value = [kw.lstrip(':') for kw in keywords_value]
keywords_value = [kw.rstrip(':') for kw in keywords_value]

# remove brakets "()"
for kc in keywords_abb_key_cand:
    kc_ = kc.lower()
    for kw in keywords_value:
        if kc_ in kw.replace('(', "").replace(')', "").split():
            kc_value = re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ')
            if (kc_ not in kc_value) and \
               (kc_value[-1] != ")") and \
               (len(kc_value.split()) > 1) and \
               ((kc not in list(keywords_abb.keys())) or (kc in list(keywords_abb.keys()) and len(kc_value) < len(keywords_abb[kc]) and len(kc_value.split()) > 1)):
                kc_value = kc_value.replace("  ", " ")
                kc_value = ' '.join(kc_value.split(' ')[:-1] + [Lem.lemmatize(kc_value.split(' ')[-1])])
                keywords_abb.update({kc:kc_value})

keywords_value = [re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ').lstrip(')').lstrip('(') for kw in keywords_value]

# remove unicodes –
unicode_keys = list(unicode_dict.keys())
for ukey in unicode_keys:
        keywords_value = [kw.replace(ukey, unicode_dict[ukey]) for kw in keywords_value]

# remove mathematics
keywords_value = [kw.replace("\\infty", "infinity") for kw in keywords_value]
keywords_value = [kw.replace("\\mathrm{", "") for kw in keywords_value]
keywords_value = [kw.replace("{", "") for kw in keywords_value]
keywords_value = [kw.replace("}", "") for kw in keywords_value]

# remove '"'
keywords_value = [kw.replace('"', "") for kw in keywords_value]

# reduce needless blanks.
keywords_value = [re.sub("\s+", " ", kw) for kw in keywords_value]
keywords_value = [kw.replace(" -", "-") for kw in keywords_value]
keywords_value = [kw.replace("- ", "-") for kw in keywords_value]
keywords_value = [kw.replace(" //", "//") for kw in keywords_value]
keywords_value = [kw.replace("// ", "//") for kw in keywords_value]

# create single-word keyword
keywords_single = [k for k in list(keywords_abb.keys())]
for kws in keywords_value:
    if len(kws.split(" ")) == 1 and len(kw) <= 4:
        keywords_single.append(kws)
    for kw in kws:
        if len(kw.split("-")) == 1 and len(kw) <= 4 and len(kw) > 1:
            keywords_single.append(kw)

keywords_single = list(np.unique(sorted(keywords_single)))

keywords_single_ = deepcopy(keywords_single)       
for kw in keywords_single_:
    if kw+"s" in keywords_single and len(kw) >= 4:
        print(kw+"s")
        keywords_single.remove(kw+"s")
    if kw[-1]+"ies" in keywords_single and kw[-1] == "y" and len(kw >=4):
        print(kw+"s")
        keywords_single.remove(kw+"s")

# create abbreviation dictionary
abb_sorted = dict(sorted(keywords_abb.items()))
with open(keyword_abb_name, "w") as j:
    json.dump(abb_sorted, j, ensure_ascii=False, indent=2)
    
# create "single" keywords list
with open(keyword_single_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords_single:
        f.write(f"{keyword}\n")

# create "value"
keywords_dict = dict(zip(keywords_raw, keywords_value))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

aerosols
aggregators
agrivoltaics
algorithms
barriers
bifurcations
buildings
characteristics
choppers
clouds
consumers
controllers
converters
correlations
costs
delays
dielectrics
diodes
dislocations
dynamics
economics
eigenvalues
electrolyzers
emissions
ensembles
experiments
failures
faults
feed-in-tariffs
forecasts
greenhouses
harmonics
heterojunctions
households
hurricanes
ibscs
igbts
imports
inverters
irradiances
measurements
metaheuristics
micro-grids
microgrids
microinverters
microturbines
mini-grids
modes
models
modules
multi-junctions
nano-grids
nanofluids
nanowires
optoelectronics
performances
perovskites
photodetectors
photodiodes
photovoltaics
prosumers
relays
renewables
resonances
scenarios
semiconductors
sensors
simulations
simulators
supercapacitors
systems
techno-economics
technoeconomics
thermodynamics
thermophotovoltaics
transients
trends
voltages
waves
wavelets


### manual refine `keywords_single`
* plural words not to be kept

In [7]:
# retrieve keywords_value
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)
keywords_value = list(keywords_dict.values())
keywords_key = list(keywords_dict.keys())

# retreive keywords_single
keywords_single = np.genfromtxt("keywords_single_refine.txt", dtype="str", delimiter="\n", encoding='utf-8-sig') #utf-8-sig for \ufeff removal
keywords_single = [k.lower() for k in keywords_single]

# Convert plural to singular
Lem = WordNetLemmatizer()
keywords_plural = {}
keywords_value_ = []
for kws in keywords_value:
    if (kws not in keywords_single) and (len(kws.split(" ")) > 1):
        kws_0 = kws.split(" ")
        
        for kws_1 in kws_0:
            kws_2 = kws_1.split("-")
            for kw in kws_2:
                if (kw not in keywords_single) and (kw != Lem.lemmatize(kw)):
                    kws = kws.replace(kw, Lem.lemmatize(kw))
                    keywords_plural.update({kw: Lem.lemmatize(kw)})
        keywords_value_.append(kws)
    else:
        keywords_value_.append(kws)

keywords_value = deepcopy(keywords_value_)
keywords_value = [kw.lstrip(' ').rstrip(' ') for kw in keywords_value]

    
# create plural-singular dictionary
plural_sorted = dict(sorted(keywords_plural.items()))
with open(keyword_plural_name, "w") as j:
    json.dump(plural_sorted, j, ensure_ascii=False, indent=2)
    
# create "value"
keywords_dict = dict(zip(keywords_key, keywords_value))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

In [8]:
# retrieve keywords_value
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)
keywords_value = list(keywords_dict.values())

# create dash dictionary
keywords_dash = []
for kw in keywords_value:
    kw_ = kw.split(" ")
    for kw__ in kw_:
        if ('-' in kw__) or ('//' in kw__):
            keywords_dash.append(kw__)
    
keywords_dash = np.unique(keywords_dash)

# create "dash" keywords dictionary - manual refinement required
keywords_dash_dict = dict(zip(keywords_dash, keywords_dash))
with open(keyword_dash_name, "w") as j:
    json.dump(keywords_dash_dict, j, ensure_ascii=False, indent=2)



### manual keywords-abb refinement
* incorrect names, typo errors
* unifying same words

### manual keyworkds-dash refinement
* incorrect names, typo errors
* unifying same words (ex. photo-voltaic and photovoltaic)

In [9]:
with open("keywords_plural.json", "r") as j:
    keywords_plural = json.load(j)

with open("keywords_abb_refine.json", "r") as j:
    keywords_abb_refine = json.load(j)

with open("keywords_dash_refine.json", "r") as j:
    keywords_dash_refine = json.load(j)
    

In [10]:
abb_refine_keys = list(keywords_abb_refine.keys())
dash_refine_keys = list(keywords_dash_refine.keys())

In [11]:
keywords_dash_refine2 = {}

plural_keys = list(keywords_plural.keys())

for dkey in dash_refine_keys:
    dvalues = keywords_dash_refine[dkey].split("; ")
    
    dvalues_abb = []
    dvalues_rms = []
    for dvalue in dvalues:
        dvalue_rem = deepcopy(dvalue)
        for akey in abb_refine_keys:
            if dvalue == akey.lower():
                dvalues.remove(dvalue)
                dvalues.append(keywords_abb_refine[akey])
            else:
                [dvalues_abb.append(keywords_abb_refine[d.upper()]) for d in dvalue.split("-") if d == akey.lower()]
                [dvalues_abb.append(keywords_abb_refine[d.upper()]) for d in dvalue.split("/") if d == akey.lower()]

    keywords_dash_refine2.update({dkey:dvalues+dvalues_abb})

with open("keywords_dash_refine2.json", "w") as j:
    json.dump(keywords_dash_refine2, j, ensure_ascii=False, indent=2)
                
            

### manual keywords_dash2 refinement
* incorrect abbs treatement

### apply keywords-dash on keywords_value
* (1) replace keywords_dash_refine.keys() to keywords_dash_refine.values()
* (2) if keywords_dash_refine2.key() has more than 2 elements, add them.

In [12]:
with open("keywords_dash_refine2.json", "r") as j:
    keywords_dash_refine2 = json.load(j)
    

keywords_value_ = []
for kws in keywords_value:
    dash_flag = 0
    kws_ = []
    for dkey in dash_refine_keys:
        
        if dkey in kws:
            kws_ += [kws.replace(dkey, keywords_dash_refine[dkey])]
            dash_flag += 1
            
            if len(keywords_dash_refine2[dkey]) > 1:
                [kws_.append(v) for v in keywords_dash_refine2[dkey][1:]]
                
    if dash_flag == 0:
        kws_ = [kws]
    
    keywords_value_.append(list(np.unique(kws_)))

keywords_dict = dict(zip(list(keywords_dict.keys()), keywords_value_))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)      

### find and extract dash and abbriviations
* create `keywords_dict_add.json`

In [13]:
%%time

import itertools

abb_refine_keys = [k.lower() for k in np.unique(list(keywords_abb_refine.keys()))]
abb_refine_values = [v.lower() for v in np.unique(list(keywords_abb_refine.values()))]

with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)    
keywords_value = list(keywords_dict.values())

keywords_value_add = []
for kw in keywords_value:
    kw_0 = kw
    kw_1 = list(itertools.chain.from_iterable([kw_.split() for kw_ in kw]))
    kw_2 = list(itertools.chain.from_iterable([kw_.split("-") for kw_ in kw]))
    kw_3 = list(itertools.chain.from_iterable([kw_.split("/") for kw_ in kw]))
    
    keywords_value_add_ = []
    
    # "dash"
    for dash_keys in dash_refine_keys:
        if dash_keys in kw_1:
            [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
        
        dash_keys_ = dash_keys.replace("-", "")
        if dash_keys_ in kw_1:
            if not (dash_keys_ in abb_refine_keys): # original abbriviation should be kept
                [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]

        dash_keys_ = dash_keys.split("-")
        if not (dash_keys_ in abb_refine_keys): # original abbriviation should be kept
            try:
                tmp = [kw_1.index(dash_key_) for dash_key_ in dash_keys_]
                if tmp == sorted(tmp) and (len(np.unique(tmp)) == len(tmp)):
                    [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
            except ValueError:
                try:
                    tmp = [kw_2.index(dash_key_) for dash_key_ in dash_keys_]
                    if tmp == sorted(tmp) and (len(np.unique(tmp)) == len(tmp)):
                        [keywords_value_add_.append(kw) for kw in keywords_dash_refine2[dash_keys]]
                except ValueError:
                    pass
    
    # "abb"
    for abb_vals in abb_refine_values:
        if abb_vals in kw and \
           (abb_vals.replace(" ", "-") not in keywords_value_add) and \
           (abb_vals.replace("-", " ") not in keywords_value_add):
            keywords_value_add_.append(abb_vals)
    
    for abb_keys in abb_refine_keys:
        if abb_keys in (kw_0 + kw_1 + kw_2 + kw_3):
            keywords_value_add_.append(keywords_abb_refine[abb_keys.upper()])
    
    keywords_value_add.append(list(np.unique(keywords_value_add_)))

# create "additional" keywords dictionary
keywords_dict_add_dashabb = dict(zip(keywords_raw, keywords_value_add))
with open("keywords_dict_add_dashabb.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)  

CPU times: user 6min 46s, sys: 78.2 ms, total: 6min 46s
Wall time: 6min 46s


### Update `keywords_manual` on `keywords_dict` and `keywords_dict_add_dashabb`

In [14]:
with open("keywords_manual.json", "r") as j:
    keywords_manual = json.load(j)
    
with open("keywords_dict_add_dashabb.json", "r") as j:
    keywords_dict_add_dashabb = json.load(j)
    
with open("keywords_dict.json", "r") as j:
    keywords_dict = json.load(j)    

In [15]:
manual_keys = list(keywords_manual.keys())
dict_values = list(keywords_dict.values())
add_dashabb_values = list(keywords_dict_add_dashabb.values())

In [16]:
dict_values_update = []

for values in dict_values:
    value_new = []
    for value in values:
        for mkey in manual_keys:
            mkey_ = mkey.lower()
            value = value.replace(mkey_, keywords_manual[mkey])
        value_new.append(value)
    dict_values_update.append(value_new)
    
keywords_dict = dict(zip(keywords_raw, dict_values_update))
with open("keywords_dict2.json", "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)        

In [17]:
add_dashabb_values_update = []

for values in add_dashabb_values:
    value_new = []
    for value in values:
        for mkey in manual_keys:
            mkey_ = mkey.lower()
            value = value.replace(mkey_, keywords_manual[mkey])
        value_new.append(value)
    add_dashabb_values_update.append(value_new)
    
keywords_dict_add_dashabb = dict(zip(keywords_raw, add_dashabb_values_update))
with open("keywords_dict_add_dashabb2.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)      

### check and remove almost same words

In [18]:
import itertools

merged = list(itertools.chain.from_iterable([dict_values_update, add_dashabb_values_update]))
merged = list(itertools.chain.from_iterable(merged))
merged_u = np.unique(merged, return_counts=True)

words_u = merged_u[0]
   

df_words = pd.DataFrame({"keywords": merged_u[0],
                         "counts": merged_u[1]
                        }).sort_values("counts", ascending=False)
df_words.head(10)

,keywords,counts
10224,photovoltaic,1731
5291,generation,257
8125,micro-grid,155
11612,pv-system,112
4287,energy-storage,108
6915,irradiance,106
5601,grid-connected,92
10982,power-point,91
7930,maximum-power-point,85
8585,mpp tracker,85


#### (1) same words with and without dash("-"): insert dash

In [19]:
keywords_dash_update = {}

count = 0
for word in words_u:
    word_wo_dash = word.replace("-", "")
    if (word_wo_dash in words_u) and ("-" in word) :
        count += 1
        
        det_0 = all([len(w) > 5 for w in word.split(" ") if "-" in w])
        det_1 = all([len(w) >= 2 for w in word.split("-")]) 
#         print(count, word, det_0, det_1)
        
        if det_0 and det_1:
            keywords_dash_update.update({word_wo_dash: word})

# print(keywords_dash_update)
print(count)

125


In [20]:
list(keywords_dash_update.keys())[:10]

['ac microgrid',
 'antireflection',
 'antireflection coating',
 'asymmetrical fault',
 'bidirectional',
 'bidirectional converter',
 'bidirectional dc/dc converter',
 'bifacial',
 'cogeneration',
 'cosimulation']

In [21]:
dash_update_keys = list(keywords_dash_update.keys())
dict_values_update_ = deepcopy(dict_values_update)

count = 0
for i, vs in enumerate(dict_values_update):
    for j, v in enumerate(vs):
        for key in dash_update_keys:
            if v == key:
                count += 1
                dict_values_update_[i][j] = keywords_dash_update[key]

print(count)
dict_values_update = deepcopy(dict_values_update_)

keywords_dict = dict(zip(keywords_raw, dict_values_update_))
with open("keywords_dict3.json", "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)      

167


In [22]:
dash_update_keys = list(keywords_dash_update.keys())
add_dashabb_values_update_ = deepcopy(add_dashabb_values_update)

count = 0
for i, vs in enumerate(add_dashabb_values_update):
    for j, v in enumerate(vs):
        for key in dash_update_keys:
            if v == key:
                count += 1
                add_dashabb_values_update_[i][j] = keywords_dash_update[key]

print(count)
add_dashabb_values_update = deepcopy(add_dashabb_values_update_)

keywords_dict_add_dashabb = dict(zip(keywords_raw, add_dashabb_values_update_))
with open("keywords_dict_add_dashabb3.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)      

39


In [23]:
merged = list(itertools.chain.from_iterable([dict_values_update_, add_dashabb_values_update_]))
merged = list(itertools.chain.from_iterable(merged))
merged_u = np.unique(merged, return_counts=True)

words_u = merged_u[0]
   

df_words = pd.DataFrame({"keywords": merged_u[0],
                         "counts": merged_u[1]
                        }).sort_values("counts", ascending=False)
df_words.head(10)

,keywords,counts
10153,photovoltaic,1731
5265,generation,257
8089,micro-grid,167
11536,pv-system,112
4266,energy-storage,108
6884,irradiance,106
5575,grid-connected,93
10907,power-point,91
7894,maximum-power-point,85
8542,mpp tracker,85


#### (2) same words with and without space(" "): remove space

In [24]:
keywords_space_update = {}

count = 0
for word in words_u:
    word_wo_space = word.replace(" ", "")
    if (word_wo_space in words_u) and (" " in word) :
        count += 1
        
        det_0 = all([len(w) > 5 for w in word.split(" ") if "-" in w])
        det_1 = all([len(w) >= 2 for w in word.split("-")]) 
#         print(count, word, det_0, det_1)
        
        if det_0 and det_1:
            keywords_space_update.update({word: word_wo_space})

# print(keywords_space_update)
print(count)

28


In [25]:
keywords_space_update

{'auto regressive': 'autoregressive',
 'back propagation': 'backpropagation',
 'co2 emission': 'co2emission',
 'dig silent': 'digsilent',
 'energy plan': 'energyplan',
 'energy plus': 'energyplus',
 'global grid': 'globalgrid',
 'hydro power': 'hydropower',
 'iec 61850': 'iec61850',
 'lab view': 'labview',
 'levenberg- marquardt': 'levenberg-marquardt',
 'light gbm': 'lightgbm',
 'mat lab': 'matlab',
 'matlab / simulink': 'matlab/simulink',
 'matlab/ simulink': 'matlab/simulink',
 'micro controller': 'microcontroller',
 'micro converter': 'microconverter',
 'micro generation': 'microgeneration',
 'micro source': 'microsource',
 'nano fluid': 'nanofluid',
 'p o': 'po',
 'perturb & observe': 'perturb&observe',
 'photo voltaic': 'photovoltaic',
 'photovoltaic/ thermal': 'photovoltaic/thermal',
 'power hardware-in-the-loop': 'powerhardware-in-the-loop',
 'ret screen': 'retscreen',
 'smart grid': 'smartgrid'}

In [26]:
with open("keywords_dict3.json", "r") as j:
    keywords_dict = json.load(j)    
    
space_update_keys = list(keywords_space_update.keys())
dict_values_update_ = deepcopy(dict_values_update)

count = 0
for i, vs in enumerate(dict_values_update):
    for j, v in enumerate(vs):
        for key in space_update_keys:
            if v == key:
                count += 1
                dict_values_update_[i][j] = keywords_space_update[key]

print(count)
dict_values_update = deepcopy(dict_values_update_)

keywords_dict = dict(zip(keywords_raw, dict_values_update_))
with open("keywords_dict3.json", "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)      

48


In [27]:
with open("keywords_dict_add_dashabb3.json", "r") as j:
    keywords_dict_dashabb3 = json.load(j)  

dash_update_keys = list(keywords_dash_update.keys())
add_dashabb_values_update_ = deepcopy(add_dashabb_values_update)

count = 0
for i, vs in enumerate(add_dashabb_values_update):
    for j, v in enumerate(vs):
        for key in dash_update_keys:
            if v == key:
                count += 1
                add_dashabb_values_update_[i][j] = keywords_dash_update[key]

print(count)
add_dashabb_values_update = deepcopy(add_dashabb_values_update_)

keywords_dict_add_dashabb = dict(zip(keywords_raw, add_dashabb_values_update_))
with open("keywords_dict_add_dashabb3.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)    

0


In [28]:
merged = list(itertools.chain.from_iterable([dict_values_update_, add_dashabb_values_update_]))
merged = list(itertools.chain.from_iterable(merged))
merged_u = np.unique(merged, return_counts=True)

words_u = merged_u[0]
   

df_words = pd.DataFrame({"keywords": merged_u[0],
                         "counts": merged_u[1]
                        }).sort_values("counts", ascending=False)
df_words.head(10)

,keywords,counts
10133,photovoltaic,1736
5260,generation,257
8072,micro-grid,167
11514,pv-system,112
4261,energy-storage,108
6876,irradiance,106
5569,grid-connected,93
10885,power-point,91
7881,maximum-power-point,85
8525,mpp tracker,85


#### (3) same words with dash("-") and space(" "): insert dash

In [29]:
keywords_dash_update = {}

count = 0
for word in words_u:
    word_wo_dash = word.replace("-", " ")
    if (word_wo_dash in words_u) and ("-" in word) :
        count += 1
        
        det_0 = all([len(w) > 5 for w in word.split(" ") if "-" in w])
        det_1 = all([len(w) >= 2 for w in word.split("-")]) 
#         print(count, word, det_0, det_1)
        
        if det_0 and det_1:
            keywords_dash_update.update({word_wo_dash: word})

# print(keywords_dash_update)
print(count)

380


In [30]:
list(keywords_dash_update.keys())[:10]

['agent based model',
 'agent based modeling',
 'air conditioning',
 'amorphous silicon',
 'arc flash',
 'back to back converter',
 'black box model',
 'black box modeling',
 'black start',
 'boost converter']

In [31]:
with open("keywords_dict3.json", "r") as j:
    keywords_dict = json.load(j)    
    
dash_update_keys = list(keywords_dash_update.keys())
dict_values_update_ = deepcopy(dict_values_update)

count = 0
for i, vs in enumerate(dict_values_update):
    for j, v in enumerate(vs):
        for key in dash_update_keys:
            if v == key:
                count += 1
                dict_values_update_[i][j] = keywords_dash_update[key]

print(count)
dict_values_update = deepcopy(dict_values_update_)

keywords_dict = dict(zip(keywords_raw, dict_values_update_))
with open("keywords_dict4.json", "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)      

796


In [32]:
with open("keywords_dict_add_dashabb3.json", "r") as j:
    keywords_dict_dashabb3 = json.load(j)  

dash_update_keys = list(keywords_dash_update.keys())
add_dashabb_values_update_ = deepcopy(add_dashabb_values_update)

count = 0
for i, vs in enumerate(add_dashabb_values_update):
    for j, v in enumerate(vs):
        for key in dash_update_keys:
            if v == key:
                count += 1
                add_dashabb_values_update_[i][j] = keywords_dash_update[key]

print(count)
add_dashabb_values_update = deepcopy(add_dashabb_values_update_)

keywords_dict_add_dashabb = dict(zip(keywords_raw, add_dashabb_values_update_))
with open("keywords_dict_add_dashabb4.json", "w") as j:
    json.dump(keywords_dict_add_dashabb, j, ensure_ascii=False, indent=2)    

371


In [34]:
merged = list(itertools.chain.from_iterable([dict_values_update, add_dashabb_values_update]))
merged = list(itertools.chain.from_iterable(merged))
merged_u = np.unique(merged, return_counts=True)

words_u = merged_u[0]
   

df_words = pd.DataFrame({"keywords": merged_u[0],
                         "counts": merged_u[1]
                        }).sort_values("counts", ascending=False)
df_words.head(10)

,keywords,counts
9924,photovoltaic,1736
5159,generation,257
7901,micro-grid,176
4187,energy-storage,120
11270,pv-system,118
7715,maximum-power-point,113
6737,irradiance,106
10144,photovoltaic-thermal,103
13329,solar-photovoltaic,96
5450,grid-connected,96


#### (4) apply `unique()` on values

In [45]:
with open("keywords_dict4.json", "r") as j:
    keywords_dict = json.load(j)   
dict_values = list(keywords_dict.values())
dict_keys = list(keywords_dict.keys())

with open("keywords_dict_add_dashabb4.json", "r") as j:
    keywords_dict_dashabb = json.load(j)  
dict_dashabb_values = list(keywords_dict_dashabb.values())

value_unique = []
for i, (dict_value, dashabb_value) in enumerate(zip(dict_values, dict_dashabb_values)):
    v1 = list(itertools.chain.from_iterable([v.split("; ") for v in dict_value]))
    v2 = list(itertools.chain.from_iterable([v.split("; ") for v in dashabb_value]))
    value_unique.append(list(set(v1 + v2)))
    
keywords_dict_u = dict(zip(keywords_raw, value_unique))
with open("keywords_dict_u.json", "w") as j:
    json.dump(keywords_dict_u, j, ensure_ascii=False, indent=2)        

### Update keywords based on `frequency` (top 1200)

In [156]:
value_u = list(itertools.chain.from_iterable(value_unique))
value_u = np.unique(value_u, return_counts=True)
df_words = pd.DataFrame({"keywords": value_u[0],
                         "counts": value_u[1]
                        }).sort_values("counts", ascending=False).reset_index(drop=True)
df_words.head(10)

,keywords,counts
0,photovoltaic,1618
1,generation,255
2,micro-grid,146
3,pv-system,111
4,energy-storage,108
5,irradiance,104
6,maximum-power-point,96
7,power-point,91
8,photovoltaic-thermal,89
9,grid-connected,89


In [157]:
def split_kw(kw):
    kw_space = kw.split(" ")
    return kw_space

df_words["single"] = df_words["keywords"].apply(split_kw)
df_words.head(10)

,keywords,counts,single
0,photovoltaic,1618,[photovoltaic]
1,generation,255,[generation]
2,micro-grid,146,[micro-grid]
3,pv-system,111,[pv-system]
4,energy-storage,108,[energy-storage]
5,irradiance,104,[irradiance]
6,maximum-power-point,96,[maximum-power-point]
7,power-point,91,[power-point]
8,photovoltaic-thermal,89,[photovoltaic-thermal]
9,grid-connected,89,[grid-connected]


In [114]:
stop_words = ["and", "of", "the"]

single_merged = list(itertools.chain.from_iterable(df_words["single"].tolist()))
single_merged_ = deepcopy(single_merged)

for i, word in enumerate(single_merged):
    if (word in abb_refine_keys) and (len(keywords_abb_refine[word.upper()].split(" "))==1):
        single_merged_[i] = keywords_abb_refine[word.upper()]

single_merged_u = np.unique(single_merged_, return_counts=True)        

df_words_single = pd.DataFrame({"keywords": single_merged_u[0],
                         "counts": single_merged_u[1]
                        }).sort_values("counts", ascending=False).reset_index(drop=True)

for sword in stop_words:
    idx = df_words_single[df_words_single["keywords"] == sword ].index
    df_words_single.drop(idx, axis=0, inplace=True)

df_words_single.head(10)

,keywords,counts
0,photovoltaic,905
1,system,788
2,power,759
3,model,677
4,energy,635
5,solar,607
6,modeling,467
8,control,350
10,converter,232
11,hybrid,224


In [116]:
np.where(df_words_single["keywords"]=="shadow")

(array([133]),)

In [121]:
np.where(df_words_single["keywords"]=="building")

(array([31]),)

In [117]:
np.where(df_words_single["keywords"]=="bipv")

(array([1151]),)

In [119]:
words_single_1200 = df_words_single["keywords"].iloc[:1200].tolist()
print(words_single_1200[:20])
print(words_single_1200[-20:])

['photovoltaic', 'system', 'power', 'model', 'energy', 'solar', 'modeling', 'control', 'converter', 'hybrid', 'analysis', 'algorithm', 'network', 'optimization', 'prediction', 'load', 'generation', 'voltage', 'cell', 'inverter']
['traffic', 'mosfet', 'volume', 'integral', 'positive', 'common-mode', 'common', 'explicit', 'arima', 'asset', 'decoupled', 'zinc', 'optoelectronic', 'phenomenon', 'porous', 'mission', '5-level', 'cavity', 'evacuated', 'assisted']


In [126]:
np.where(df_words_single["counts"]>30)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [131]:
words_single_180 = df_words_single["keywords"].iloc[:180].tolist()

#### extract words in top 180 + bipv

In [152]:
words_manual = ["bipv"]
words_select = words_single_180 + words_manual

freq_values = []
for values in value_unique:
    values_add = []
    for w in words_select:
        values_add.append([w for v in values if w in v.replace("-", " ").split(" ")])
    freq_values.append(list(set(itertools.chain.from_iterable(values_add))))

print(len(freq_values))
keywords_dict_freq = dict(zip(keywords_raw, freq_values))
with open("keywords_dict_freq.json", "w") as j:
    json.dump(keywords_dict_freq, j, ensure_ascii=False, indent=2)      

19457


In [153]:
value_unique[900:910]

[['behavioural policy'],
 ['behind-the-meter solar', 'behind-the-meter'],
 ['behind-the-meter'],
 ['behind-the-meter',
  'pv-storage',
  'behind-the-meter pv and storage',
  'photovoltaic'],
 ['behind-the-meter', 'solar-pv', 'photovoltaic', 'behind-the-meter solar pv'],
 ['belief'],
 ['benchmark model'],
 ['benchmark network'],
 ['benchmarking'],
 ['benchmarking comparison']]

In [154]:
freq_values[900:910]

[[],
 ['solar'],
 [],
 ['photovoltaic', 'storage'],
 ['solar', 'photovoltaic'],
 [],
 ['model'],
 ['network'],
 [],
 []]

In [155]:
np.where(df_words_single["keywords"]=="mode")

(array([90]),)

### importing "knowledge"

In [3]:
keywords_knowledge_init = {
    "statistics": [
        "analysis_of_variance", 
        "autoregressive_integrated_moving_average", 
        "autoregressive_moving_average", 
        "markov", 
        "kalman_filter", 
        "bayes", 
        "gaussian_process", 
        "autoregressi", 
        "probabilistic", 
        "kfold", 
        "ensemble", 
        "kruskal_wallis" 
    ], 

    "method": [
        "taguchi", 
        "averaging_point_method", 
        "describing_function_method", 
        "finite_difference_time_domain_method", 
        "finite_element_method", 
        "group_method_of_data_handling", 
        "incremental_conductance_method", 
        "match_evaluation_method", 
        "multiple_shifted_frequency_method", 
        "numerical_method", 
        "oblique_asymptote_method", 
        "response_surface_methodology", 
        "steepest_descent_method", 
        "nelder_mead", 
        "newton_raphson", 
        "runge_kutta", 
        "conditional_interpolation", 
        "expectation_maximization", 
        "dynamic_simulation", 
        "empirical", 
        "emulation", 
        "gradient_descent", 
        "algorithm" 
    ],

    "metric": [
        "mean_square_error", 
        "mean_absolute_error", 
        "mean_absolute_percentage_error", 
        "cross_entropy", 
        "least_square"
    ], 

    "machine_learning": [
        "clustering", 
        "regression", 
        "classification", 
        "dimension_reduction", 
        "reinforcement_learning", 
        "ensemble", 
        "natural_language_processing"
    ], 

    "doc2vec":[
        "natural_language_processing"
    ], 

    "reinforcement_learning": [
        "sarsa", 
        "markov"
    ],
    
    "ensemble": [
        "random_forest", 
        "adaboost",
        "bagging",
        "bootstrap", 
        "lightgbm", 
        "xgboost"
    ], 
    
    "data": [
        "database",
        "data_acquisition",
        "data_mining",
        "data_driven",
        "data_based"
    ],
    
    "database": [
        "mapreduce",
        "sql",
        "hadoop"
    ],
    
    "classification": [
        "k_nearest_neighbor",
        "support_vector_machine",
        "neural_network"
    ],
    
    "regression": [ 
        "k_nearest_neighbor",
        "support_vector_machine",
        "neural_network",
        "ridge",
        "lasso",
        "autoregressive",
        "support_vector_regression"
    ],
    
    "clustering": [
        "k_means", 
        "dbscan"
    ],
    
    "neural_network": [
        "artificial_neural_network",
        "learning_vector_quantization",
        "recurrent_neural_network",
        "convolutional_neural_network",
        "autoencoder",
        "adaline",
        "artificial_neural_fuzzy_inference_system",
        "elman",
        "attention",
        "extreme_learning_machine",
        "multilayer_perceptron",
    ],
    
    "recurrent_neural_network": [
        "long_short_term_memory",
        "boltzmann_machine",
    ],
    
    "convolutional_neural_network": [
        "googlenet"
    ], 

    "dimension_reduction": [
        "principal_component_analysis",
        "factor_analysis",
        "autoencoder"
    ],
    
    "algorithm": [
        "ant_colony", 
        "ant_lion", 
        "artificial_bee_colony", 
        "artificial_fish_swarm", 
        "backtracking_search", 
        "bacterial_foraging", 
        "bat", 
        "bee_pollinator", 
        "binary_search", 
        "bio_inspired", 
        "bucket_elimination", 
        "crow_search", 
        "elite_retention", 
        "evolutionary", 
        "firefly", 
        "fireworks_explosion", 
        "flower_pollination", 
        "fruitfly", 
        "genetic_algorithm", 
        "golden_section", 
        "grasshopper", 
        "gravity_search", 
        "grey_wolf", 
        "imperialist_competition", 
        "jaya", 
        "leapfrog", 
        "honey_bee_mating", 
        "interior_search", 
        "invasive_weed", 
        "elephant_herding", 
        "particle_swarm", 
        "pattern_search", 
        "perturb_and_observe", 
        "shuffled_frog_leaping", 
        "versatile_threshold", 
        "monte_carlo", 
        "rule_based", 
        "dynamic_programming"
    ],
    
    "dynamic_programming": [
        "reinforcement_learning"
    ]
}

with open("keywords_knowledge_init.json", "w") as j:
    json.dump(keywords_knowledge_init, j, ensure_ascii=False, indent=2) 

In [4]:
kn_keys_init = list(keywords_knowledge_init.keys())
kn_values_init = list(keywords_knowledge_init.values())

In [6]:
kn_values_all = list(itertools.chain.from_iterable(kn_values_init))

In [ ]:
# DFS using stack

for k in kn_values_all:
    k_start = deepcopy(k)
    stack = []
    
    for idx_v, v in enumerate(kn_keys_init):
        if k in keywords_knowledge_init[v]:
            stack.append(idx_v)
            if k_start == "xgboost":
                print(k_start, k, v, idx_v, stack)
            
            k = v

In [17]:
kn_values = []

for v in kn_values_all:
    v_ = deepcopy(v)
    value_final = []
    for k in kn_keys_init:
        k_ = deepcopy(k)
        for i in range(10):
            try:
                if v_ in keywords_knowledge_init[k_]:
                    if v_ == "xgboost":
                        print(f"k_, v: {k_}, {v}")
                    value_final.append(k_)
                    v_ = k_
                    if v == "xgboost":
                        print(f"new v_: {v_}")
            except KeyError:
                kn_values.append(value_final)
                if v == "xgboost":
                    print(f"# v, value_final: {v}, {value_final}\n")
                break
            
            

k_, v: ensemble, xgboost
new v_: ensemble


In [ ]:
!pwd

In [15]:
kn_values = []

for key in kn_keys:
    value = []
    key_ = deepcopy(key)
    for i in range(10):
        try:
            if key == "markov":
                print(key_)
            key_ = keywords_knowledge_init[key_]
            value.append(key_)
        except KeyError:
            kn_values.append(value)
#             print(key, value)
            break
    

markov
reinforcement_learning
dynamic_programming
algorithm
method
